А в разделе 7.1 мы использовали функциональный API Keras для тренировки (на задаче предсказания случайно сгенерированных данных).
Последовательная модель, модель с несколькими входами, модель с несколькими выходами, реализация модуля Inception.
Повторное использование слоев и модели как слои.

In [1]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)
model.summary()

Using TensorFlow backend.





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [2]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_train, y_train)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





1000/1000 [==============================] - 0s 347us/step - loss: 11.6020
Epoch 2/10
1000/1000 [==============================] - 0s 21us/step - loss: 11.5568
Epoch 3/10
1000/1000 [==============================] - 0s 22us/step - loss: 11.5470
Epoch 4/10
1000/1000 [==============================] - 0s 17us/step - loss: 11.5418
Epoch 5/10
1000/1000 [==============================] - 0s 23us/step - loss: 11.5371
Epoch 6/10
1000/1000 [==============================] - 0s 15us/step - loss: 11.5344
Epoch 7/10
1000/1000 [==============================] - 0s 22us/step - loss: 11.5317
Epoch 8/10
1000/1000 [==============================] - 0s 18us/step - loss: 11.5297
Epoch 9/10
1000/1000 [==============================] - 0s 17us/step - loss: 11.5278
Epoch 10/10
1000/1000 [==============================] - 0s 50us/step


In [4]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500
text_input = Input(shape=(None,), dtype='int32', name='text')

embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [6]:
import numpy as np
num_samples = 1000
max_length = 100
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.zeros(shape=(num_samples, answer_vocabulary_size))
indices = np.random.randint(0, answer_vocabulary_size, size=num_samples)
for i, x in enumerate(answers):
    x[indices[i]] = 1
model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.3557 - acc: 0.0040
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.1456 - acc: 0.0080
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.0594 - acc: 0.0070
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.9349 - acc: 0.0090
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.8750 - acc: 0.0120
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.7514 - acc: 0.0180
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.6395 - acc: 0.0210
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.5528 - acc: 0.0260
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.4809 - acc: 0.0280
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.4070 - acc: 0.0240


In [7]:
from keras import layers
from keras import Input
from keras.models import Model
vocabulary_size = 50000
num_income_groups = 10
posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)
model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'], loss_weights=[0.25, 1., 10.])
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

NameError: name 'posts' is not defined

In [8]:
from keras import layers
branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)
branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)
branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)
branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

ValueError: Input 0 is incompatible with layer conv2d_1: expected ndim=4, found ndim=2

In [10]:
from keras import layers
from keras import Input
from keras.models import Model
lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)
merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)
model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)

NameError: name 'left_data' is not defined

In [11]:
from keras import layers
from keras import applications
from keras import Input
xception_base = applications.Xception(weights=None, include_top=False)
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))
left_features = xception_base(left_input)
right_input = xception_base(right_input)
merged_features = layers.concatenate([left_features, right_input], axis=-1)